In [93]:
import pandas as pd
import numpy as np
from datetime import date
from statistics import mean, mode
from pandas.api.types import CategoricalDtype
pd.set_option('display.max_rows', None)

In [2]:
today = date.today()
print(today)

2023-10-20


# Tables

In [3]:
path_tables = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 2 tables/"

In [4]:
# tables
filename_careerorglink = "careerorglink.xlsx"
filename_leadercareerlink = "leadercareerlink.xlsx"
filename_orgtree = "orgtree.xlsx"
filename_elected = "positions_elected.xlsx"

In [5]:
col = pd.read_excel(path_tables + filename_careerorglink,dtype="str")
col.shape

(9002, 12)

In [6]:
col.columns

Index(['CareerString', 'CareerDateString_2022', 'IsJob', 'MultipleSubstrings',
       'CareerStartYear', 'CareerStartMonth', 'CareerSubstring',
       'InstitutionType', 'PrimaryInstitution', 'OrgName', 'Position',
       'Notes'],
      dtype='object')

In [7]:
lcl = pd.read_excel(path_tables + filename_leadercareerlink,dtype="str")
lcl.shape

(12617, 3)

In [8]:
lcl.columns

Index(['LeaderID', 'CareerString', 'CareerDateString_2022'], dtype='object')

In [9]:
org = pd.read_excel(path_tables + filename_orgtree,dtype="str")
org.shape

(2368, 19)

In [10]:
org.columns

Index(['InstitutionType', 'OrgType', 'PrimaryInstitution', 'OrgName',
       'PI_Index', 'OrgRank', 'P1', 'P2', 'P3', 'Alias_OrgName',
       'LinkToNext_PI', 'LinkToNext_Org', 'LinkToNext_Year', 'Notes',
       'L1_Index', 'L2_Index', 'L3_Index', 'L4_Index', 'L5_Index'],
      dtype='object')

In [11]:
elected = pd.read_excel(path_tables + filename_elected,dtype="str")
elected.shape

(105, 4)

In [12]:
elected.columns

Index(['PrimaryInstitution', 'OrgName', 'Position', 'IsElected'], dtype='object')

# Queries

In [13]:
path_queries = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 3 queries/"

In [14]:
# jobs and job transitions, elected & unelected, in and out of gov
filename_leaderjob_electUnelect_inOutgov = "leaderjob_electUnelect_inOutgov.xlsx"
filename_leaderjobtransition_electUnelect_inOutgov = "leaderjobtransition_electUnelect_inOutgov.xlsx"

In [28]:
# certain jobs
# IsJob == True
# InstitutionType, PrimaryInstitution, OrgName != "Uncertain"
jobs = pd.read_excel(path_queries + filename_leaderjob_electUnelect_inOutgov,dtype="str")
jobs.shape

(8594, 14)

In [15]:
trans = pd.read_excel(path_queries + filename_leaderjobtransition_electUnelect_inOutgov,dtype="str")
trans.shape

(6311, 29)

#### change datatypes

In [16]:
trans = trans.astype({"OrgAdvance":"int","PositionAdvance":"int","CareerStartYear_1":"int"})
trans.dtypes

LeaderID                   object
CareerString_1             object
CareerDateString_2022_1    object
CareerStartYear_1           int32
CareerStartMonth_1         object
CareerStartDate_1          object
CareerSubstring_1          object
InstitutionType_1          object
PrimaryInstitution_1       object
OrgName_1                  object
Position_1                 object
IsElected_1                object
OrgRank_1                  object
PositionRank_1             object
CareerString_2             object
CareerDateString_2022_2    object
CareerStartYear_2          object
CareerStartMonth_2         object
CareerStartDate_2          object
CareerSubstring_2          object
InstitutionType_2          object
PrimaryInstitution_2       object
OrgName_2                  object
Position_2                 object
IsElected_2                object
OrgRank_2                  object
PositionRank_2             object
OrgAdvance                  int32
PositionAdvance             int32
dtype: object

# Functions

In [17]:
def merge_results(m):
    
    print("\nMerge Results...")
    print("")
    print("\tshape     :",m.shape)
    print("\tleft_only :",m[m["_merge"]=="left_only"].shape)
    print("\tboth      :",m[m["_merge"]=="both"].shape)
    print("\tright_only:",m[m["_merge"]=="right_only"].shape)

In [18]:
# using this on (PI,OrgName) will ensure unique & non-null keys
# using this on a larger df will ensure unique rows and non-null keys, but not unique keys

def unique_non_null_rows(olddf):
    
    df = olddf.copy()
    
    
    ### drop duplicates
    df = df.drop_duplicates(keep="first",ignore_index=True)

    ### drop null rows
    df.dropna(how="all",axis=0,inplace=True)
    
    # drop rows with null PI
    df = df[~(df["PrimaryInstitution"].isna())]
        
    ### drop rows where PI contains stop words
    stop_words_lower = ["uncertain","current","deprecated","please_revise"]
    df = df[ ~ df["PrimaryInstitution"].str.lower().isin(stop_words_lower)]
            

    print("\nUnique Non-null Rows...")
    print("")
    print("\tNon-unique rows:",olddf.shape)
    print("\tUnique rows    :",df.shape)

    ### sort
    df = df.sort_values(["PrimaryInstitution","OrgName"])
    
    return df

In [19]:
def create_time_series(series,group_var,count_var):
    
    yeardist = series.groupby(group_var,as_index=False).count().sort_values(group_var)
    
    max_year = max(series[group_var])
    x = pd.DataFrame({"year":pd.Series(range(min(yeardist[group_var].astype(int)),max(yeardist[group_var].astype(int))+1))})
    
    yeardist[group_var] = yeardist[group_var].astype(int,errors="ignore")
    
    ts = x.merge(yeardist,left_on="year",right_on=group_var,how="left")
    ts.loc[ts[count_var].isna(),count_var]=0
    # merge_results(ts)
    
    ts_columns = ["year",count_var]
    ts = ts[ts_columns]
    
    return ts

# Analysis - Research Note

In [22]:
path_analysis = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/analysis/"

In [23]:
# analysis sub-paths
study0_path = "2023.10.04 Study 0 - research note/"
study1_path = "2023.10.04 Study 1 - political capital/"
study2_path = "2023.10.04 Study 2 - commitment vs control/"
study3_path = "2023.10.04 Study 3 - reds vs experts/"

# Descriptive Figures

### job count, by InstitutionType

In [56]:
figure_columns = ["InstitutionType","CareerString"]
figure_column_names = ["InstitutionType","InstitutionTypeCount"]
fig1 = jobs[figure_columns].groupby("InstitutionType",as_index=False).count().sort_values("CareerString",ascending=False)
fig1.columns = figure_column_names
fig1.reset_index(drop=True,inplace=True)
fig1

,InstitutionType,InstitutionTypeCount
0,정권기관,4577
1,노동당,2282
2,인민군,548
3,국제친선단체,257
4,당외곽및사회단체_사회부문(별책),223
5,당외곽및사회단체_정치부문,133
6,당외곽및사회단체_사회부문,121
7,당외곽및사회단체_경제부문(별책),102
8,당외곽및사회단체_근로단체,100
9,당외곽및사회단체_체육부문,94


In [57]:
ingov = ["정권기관","노동당","인민군"]

In [68]:
fig1_short = fig1.copy()
fig1_short.loc[~fig1_short.InstitutionType.isin(ingov),"InstitutionType"]="당외곽및사회단체"
fig1_short = fig1_short.groupby("InstitutionType",as_index=False).agg({"InstitutionTypeCount":"sum"})
fig1_short

,InstitutionType,InstitutionTypeCount
0,노동당,2282
1,당외곽및사회단체,1187
2,인민군,548
3,정권기관,4577


In [70]:
fig1name = "Job Count by InstitutionType.xlsx"
fig1_short.to_excel(path_analysis + study0_path + fig1name,index=False)

### job count, by Elected

In [86]:
jobs.loc[jobs["IsElected"].isna(),"IsElected"]=0

In [90]:
jobs[jobs.PrimaryInstitution=="최고인민회의"]

,LeaderID,CareerString,CareerDateString_2022,CareerStartYear,CareerStartMonth,CareerStartDate,CareerSubstring,InstitutionType,PrimaryInstitution,OrgName,Position,IsElected,OrgRank,PositionRank
4,김창섭,최고인민회의 제13기 대의원,2014.03,2014,3,201403,NaN,정권기관,최고인민회의,NaN,대의원,1,0,3
5,최태복,최고인민회의 제13기 대의원,2014.03,2014,3,201403,NaN,정권기관,최고인민회의,NaN,대의원,1,0,3
6,장창하,최고인민회의 제14기 대의원,2019,2019,NaN,201900,NaN,정권기관,최고인민회의,NaN,대의원,1,0,3
9,최희태,(前)모란봉구역 인민위원회 위원장,2013.12,2013,12,201312,NaN,정권기관,최고인민회의,평양시인민위원회_모란봉구역인민위원회,위원장,0,2,1
10,차희림,(前)법제위원회 위원,2014.04,2014,4,201404,NaN,정권기관,최고인민회의,법제위원회,위원,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8569,최광,황해남도 인민위원회 위원장(김일성 황해남도 현지지도시 사면),1977.04,1977,4,197704,NaN,정권기관,최고인민회의,황해남도인민위원회,위원장,0,1,1
8579,박용팔,황해북도 서흥군 인민위원회 위원장,2007.08,2007,8,200708,NaN,정권기관,최고인민회의,황해북도인민위원회_서흥군인민위원회,위원장,0,2,1
8580,리상관,황해북도 인민위원회 위원장 * 전임: 김병송,2005.04,2005,4,200504,NaN,정권기관,최고인민회의,황해북도인민위원회,위원장,0,1,1
8581,강영수,황해북도 인민위원회 위원장 * 전임: 리원일,2012.09,2012,9,201209,NaN,정권기관,최고인민회의,황해북도인민위원회,위원장,0,1,1


In [92]:
figure_columns = ["IsElected","CareerString"]
fig2 = jobs[figure_columns].groupby("IsElected",dropna=False,as_index=False).count()
fig2

,IsElected,CareerString
0,0,4413
1,1,4181


In [97]:
elected_columns = ["InstitutionType","PrimaryInstitution","OrgName","Position","CareerString"]
elected_jobs_count = jobs.loc[jobs.IsElected=="1",elected_columns].groupby(["InstitutionType","PrimaryInstitution","OrgName","Position"],dropna=False,as_index=False).count().sort_values(["InstitutionType","PrimaryInstitution","CareerString"],ascending=False)
elected_jobs_count

,InstitutionType,PrimaryInstitution,OrgName,Position,CareerString
73,정권기관,평양시인민위원회,NaN,위원장,11
72,정권기관,평양시인민위원회,NaN,부위원장,1
67,정권기관,최고인민회의,NaN,대의원,1998
49,정권기관,최고인민회의,상임위원회,위원,99
41,정권기관,최고인민회의,법제위원회,위원,51
63,정권기관,최고인민회의,중앙선거위원회,위원,36
52,정권기관,최고인민회의,예산위원회,위원,35
44,정권기관,최고인민회의,상설회의,의원,20
34,정권기관,최고인민회의,대의원자격심사위원회,위원,18
48,정권기관,최고인민회의,상임위원회,서기장,16


In [98]:
# export elected_jobs_count
elected_jobs_count_fname = "Elected Job Counts.xlsx"
elected_jobs_count.to_excel(path_analysis + study0_path + elected_jobs_count_fname,index=False)

In [99]:
figure_column_names = ["Is Elected","Job Count"]
fig2.columns = figure_column_names
fig2.reset_index(drop=True,inplace=True)
fig2

,Is Elected,Job Count
0,0,4413
1,1,4181


In [100]:
# export figure 2
fig2_fname = "Job Counts by IsElected.xlsx"
fig2.to_excel(path_analysis + study0_path + fig2_fname,index=False)

### job counts by